# Create Enrichment Map directly from R

-------------------

**Table of Contents:**
  1. <a href=#launch>Launch Cytoscape</a>
  1. <a href=#test>Test connection from R to cytoscape/a>
  1. <a href=#create>Create Enrichment Map</a>
  1. <a href=#examine>Examine results</a>

-------------------

<a id='launch'></a>

## 1. Launch Cytoscape

Create EM through Cyrest interface - make sure you open cytoscape with a -R 1234 (to enable rest functionality) and allow R to talk directly to cytoscape.

**Launch Cytoscape**

On **Windows** open a command window and run:
cd "C:\Program files\Cytoscape_v3.3.0\" \ 
    cytoscape.bat -R 1234
    
On **MAC** open a terminal window and run:
/Applications/Cytoscape_v3.3.0/cytoscape.sh -R 1234

><span style="color:red">**the code below can not be run from within the docker container of this tutorial unless you have mapped the localhost to the ip address of the computer you are running the docker image from.  Add the following to the docker run command (substitute your own ip address though)  --add-host="localhost:192.168.0.10" .  **</span>
* If you have cytoscape and R running on your computer you can run it directly from R. The R code is basically constructing a url that it then calls in order to create the network in cytoscape.  

<a id='test'></a>

## 2. Set up connection from R to cytoscape 

In [15]:
library(RJSONIO)

library(httr)
# Basic settings
port.number = 1234
base.url = paste("http://localhost:", toString(port.number), "/v1", sep="")

print(base.url)

version.url = paste(base.url, "version", sep="/")
cytoscape.version = GET(version.url)
cy.version = fromJSON(rawToChar(cytoscape.version$content))
print(cy.version)

[1] "http://localhost:1234/v1"
      apiVersion cytoscapeVersion 
            "v1"          "3.3.0" 


<a id='create'></a>

## 3. Create Enrichment Map

<span style="color:blue">**Specify the path to your data directory. Change variable "path_to_file" **</span><br> 
<span style="color:red">**On windows use / instead of the regular \ to specify the path.!**</span> 

In [16]:

# to create an Enrichment map we need to specify
# analysisType = generic
# 
enrichmentmap.url <- paste(base.url, "commands","enrichmentmap","build", sep="/") 

path_to_file="/Users/risserlin/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Jupyter_Notebooks/notebooks/data/"

#on windows change the \ to / in order for the pathname to be interpretted correctly.
#path_to_file="C:/Users/zaphod/Ruth_dropbox/Dropbox (Bader Lab)/Ruth Isserlin's files/Enrichment_Analyses/Jupyter_Notebooks/notebooks/data/"

enr_file = paste(path_to_file,"gprofiler_results_mesenonly_ordered_computedinR.txt",sep="")

em_params <- list(analysisType = "generic",enrichmentsDataset1 = enr_file,pvalue="1.0",qvalue="0.00001",
                  #expressionDataset1 = exp_file, 
                  similaritycutoff="0.25",coeffecients="JACCARD")

response <- GET(url=enrichmentmap.url, query=em_params)

In [17]:
#get the url used to generate network.  
response$url

[1] "http://localhost:1234/v1/commands/enrichmentmap/build?analysisType=generic&enrichmentsDataset1=%2FUsers%2Frisserlin%2FDropbox%20%28Bader%20Lab%29%2FRuth%20Isserlin%27s%20files%2FEnrichment_Analyses%2FJupyter_Notebooks%2Fnotebooks%2Fdata%2Fgprofiler_results_mesenonly_ordered_computedinR.txt&pvalue=1.0&qvalue=0.00001&similaritycutoff=0.25&coeffecients=JACCARD"

**Cut and Copy the above URL into a web browser if you haven't bound your machine IP to the docker image (as described above)**

In [18]:
#get the content returned.  If the call was successful the message should be "finished"
 content(response, "text", encoding = "ISO-8859-1")

[1] "Finished\n"

If executed successfully above message should be "Finished"

<a id='examine'></a>

## 4. Examine results

**Go to your open instance of cytoscape to see you results. **<BR>
Network should look similar to the below figure

<img src="figures/gprofiler_example_network_forjupyter.png">